In [4]:
import pandas as pd
df = pd.read_csv('/Users/oussamanaji/Downloads/oussama_2025_01_14.csv')
colonnes = ['outcome','ecart','created_at', 'phone_number']
df = df[colonnes]
df['ecart'] = pd.to_timedelta(df['ecart'])
df = df[(df['ecart'] > pd.Timedelta(0)) & (df['ecart'] < pd.Timedelta(minutes=40)) & (df['outcome'] != 'Waiting for lead update')]
df

,outcome,ecart,created_at,phone_number
0,Qualified,0 days 00:25:16.066000,2025-01-08 15:40:18+00:00,33145683129
2,Unsuccessful Contact Attempts,0 days 00:23:27.425000,2024-12-24 00:00:09+00:00,33175914536
3,Unsuccessful Contact Attempts,0 days 00:23:07.861000,2024-11-09 13:04:32+00:00,33601041363
4,Unsuccessful Contact Attempts,0 days 00:23:32.942000,2024-11-25 14:34:06+00:00,33601375425
5,Qualified,0 days 00:21:35.447000,2024-11-28 14:06:02+00:00,33601653785
...,...,...,...,...
2330,Qualified,0 days 00:25:12.582000,2024-12-23 10:42:25+00:00,33782548820
2331,Lead’s own initiative,0 days 00:19:42.918000,2024-12-04 17:17:53+00:00,33783598549
2332,Unsuccessful Contact Attempts,0 days 00:26:19.230000,2024-12-09 13:11:17+00:00,33785398006
2333,Conversation denied,0 days 00:20:23.878000,2024-11-25 14:37:17+00:00,33785844641


In [5]:
df['outcome'].unique()

array(['Qualified', 'Unsuccessful Contact Attempts',
       'Conversation denied', 'Pending lead answer', 'Purchase Cancelled',
       'Already bought a car', 'Invalid Data', 'Lead’s own initiative',
       nan], dtype=object)

In [3]:
df_appels_bruts = pd.read_csv('/Users/oussamanaji/Downloads/appels_brut.csv')
df_appels_bruts.head()

,Date/Heure,Numéro,Destination
0,2025-01-01 12:44:50,33651897703,France Mobile Free Mobile
1,2025-01-01 12:25:20,33632323619,France Mobile Orange
2,2025-01-01 12:20:16,33629612710,France Mobile SFR
3,2025-01-01 11:58:28,33235937936,France
4,2025-01-01 11:47:46,33695966064,France Mobile Free Mobile


In [7]:
# Joindre les deux DataFrames sur phone_number et Numéro
df_combined = pd.merge(df, df_appels_bruts, left_on='phone_number', right_on='Numéro')

# Filtrer les outcomes pertinents
df_filtered = df_combined[df_combined['outcome'].isin(['Qualified', 'Unsuccessful Contact Attempts'])]

# Grouper par opérateur et outcome, puis compter
counts = df_filtered.groupby(['Destination', 'outcome']).size().unstack(fill_value=0)

# Calculer le taux de 'Qualified'/'Unsuccessful Contact Attempts'
counts['Qualified_to_Unsuccessful_Ratio'] = counts['Qualified'] / counts['Unsuccessful Contact Attempts']

# Remplacer les infinities et NaN par 0 si nécessaire
counts = counts.fillna(0).replace([float('inf'), -float('inf')], 0)

# Afficher le résultat
counts[['Qualified', 'Unsuccessful Contact Attempts', 'Qualified_to_Unsuccessful_Ratio']]

outcome,Qualified,Unsuccessful Contact Attempts,Qualified_to_Unsuccessful_Ratio
Destination,,,
France,5,10,0.500000
France Mobile,1,5,0.200000
France Mobile Bouygues,62,434,0.142857
France Mobile Free Mobile,85,398,0.213568
France Mobile Lycatel,69,242,0.285124
France Mobile NRJ,7,58,0.120690
France Mobile Orange,37,557,0.066427
France Mobile SFR,157,815,0.192638
